In [1]:
# import os
# import pandas as pd

# BASE_DIR = "../llama_profiling/experiments/llama_profiling"
# OUTPUT_FILE = "merged_energibridge.csv"

# csv_files = [
#     os.path.join(root, f)
#     for root, _, files in os.walk(BASE_DIR)
#     for f in files if f.endswith(".csv")
# ]

# dfs = []
# for path in csv_files:
#     df = pd.read_csv(path)
#     df.columns = df.columns.str.strip()
#     dfs.append(df)

# merged = pd.concat(dfs, ignore_index=True)

# merged.to_csv(OUTPUT_FILE, index=False)
# print(f"Merged CSV saved: {OUTPUT_FILE}")

import os
import pandas as pd
import re
import numpy as np

In [12]:
BASE_DIR = "../llama_profiling/experiments/llama_profiling"
OUTPUT_FILE = "merged_energibridge.csv"

csv_files = [
    os.path.join(root, f)
    for root, _, files in os.walk(BASE_DIR)
    for f in files if f.endswith(".csv")
][1:2]

dfs = []

results = []

run_table_pd = pd.read_csv('../llama_profiling/experiments/llama_profiling/run_table.csv')


def split_run_id(val):
    # Search for patterns like "run_0_repetition_0" or "run0_repetition0"
    match = re.search(r'run_?(\d+)_+repetition_?(\d+)', val)
    if match:
        run_num, repetition_num = match.groups()
        return run_num, repetition_num
    return None, None
    
for i,path in enumerate(csv_files):
    folder = path[:path.rfind("\\")]
    #if 'run_0_repetition_15' not in path:
    #    print('ignoring', path)
    #    continue

    print('running', path)
    df = pd.read_csv(path)
    df.columns = df.columns.str.strip()
    
    
    # .tsv data
    tsv_df = pd.read_csv(folder+"\\prompts_out.tsv", sep="\t")
    
    tsv_df['prompt_length'] = tsv_df["prompt"].str.len()
    tsv_df['response_length'] = tsv_df["response"].str.len()
    
    timestamp_start_prompts, timestamp_end_prompts = tsv_df.iloc[0]['start'], tsv_df.iloc[-1]['end']

    # Filter out the rows that're wihin the execution window (skipping engine initialization). 1000 is due to UNIX timestamp granularity
    df = df[(df['Time'] / 1000 >= timestamp_start_prompts) & (df['Time']  / 1000 <= timestamp_end_prompts)]
        
    # Fix overflow and normalize to energy_diff
    diff = df["PACKAGE_ENERGY (J)"] - df["PACKAGE_ENERGY (J)"].shift(1)
    df["energy_diff"] = np.where(diff < 0, df["PACKAGE_ENERGY (J)"], diff.fillna(0))
    
    # Calculate GPU consumption
    df["gpu_joules"] = df["GPU0_POWER (mWatts)"] / 5000 + df["energy_diff"]

    # Display all rows
    #with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    #    print(df[["gpu_joules", "PACKAGE_ENERGY (J)"]], )
        
    # Total consumption
    energy_consumption_total=sum(df["gpu_joules"]) + sum(df["energy_diff"])
    
    run, rep = split_run_id(path)
    
    row = {
        "run": run,
        "rep": rep,
        "duration_ms": df.iloc[-1]['Time']-df.iloc[0]['Time'],
        "cpu_joules": sum(df["energy_diff"]),
        "gpu_joules": sum(df["gpu_joules"]),
        "energy_consumption_total_joules": energy_consumption_total,
        "response_length_sum_characters": sum(tsv_df['response_length']),
        "response_token_count": sum(tsv_df['response_tokens']),
        "accuracy": sum(tsv_df['accuracy']) / 5.0
    }
    run_id = f"run_{run}_repetition_{rep}"
    
    # Display all rows
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(tsv_df[["accuracy"]], sum(tsv_df['accuracy'], skipna=Tr) / 5.0)
    
    run_table_pd.loc[run_table_pd['__run_id'] == run_id, list(row.keys())] = list(row.values())
        

run_table_pd = run_table_pd.drop('energy', axis=1)
run_table_pd.to_csv('out.csv')
print('Finished, output to file out.csv')

    


running ../llama_profiling/experiments/llama_profiling\run_0_repetition_0\energibridge.csv
    accuracy
0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
5        NaN
6        NaN
7        NaN
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14       NaN
15       NaN
16       NaN
17       NaN
18       NaN
19       NaN
20       NaN
21       NaN
22       1.0
23       NaN
24       NaN
25       NaN
26       NaN
27       0.5
28       NaN
29       NaN
30       NaN
31       NaN
32       NaN
33       NaN
34       NaN
35       NaN
36       NaN
37       NaN
38       NaN
39       NaN
40       NaN
41       NaN
42       NaN
43       NaN
44       NaN
45       NaN
46       1.0
47       NaN
48       NaN
49       NaN
50       1.0
51       NaN
52       NaN
53       NaN
54       1.0
55       NaN
56       NaN
57       NaN
58       NaN
59       NaN
60       NaN
61       NaN
62       NaN
63       NaN
64       NaN
65       NaN
66       NaN
67       NaN
68       NaN

In [8]:
dff = tsv_df[0:1]

In [9]:
dff

,start,end,prompt,response,prompt_length,response_length
0,1760502188,1760502190,Passage: Persian language -- Persian (/ˈpɜːrʒə...,"Based on the given passage, it can be said t...",696,799
